In [12]:
import sys
import numpy as np
import pandas as pd
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

print('os name:', os.name)
print('python ver:', sys.version)
print('numpy ver:', np.__version__)
print('pandas ver:', pd.__version__)
print('matplotlib ver:', matplotlib.__version__)
print('seaborn ver:', sns.__version__)

os name: nt
python ver: 3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]
numpy ver: 1.23.4
pandas ver: 1.5.0
matplotlib ver: 3.6.0
seaborn ver: 0.12.1


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 데이터 전처리

In [ ]:
# Target 균등 분포 확인(특히 분류 분석 시)
value_counts(normalize=True)

# Correlation 확인
df.corr(numeric_only=True).round(3)
df.iloc[:, :-1].corr()  # 가장 오른쪽에 있는 Target값의 dtype이 object인 경우 제외시키기 위해

sns.heatmap(df.corr(numeric_only=True))

## 데이터 준비

In [ ]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
print(X.shape, Y.shape)  # Row 개수가 동일하지 확인 (Row, Col)

## 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.preprocessing import PolynomialFeatures
degree = 3
P = PolynomialFeatures(degree=degree, include_bias=False)
X_poly = P.fit_transform(X)

## 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

- test_size = 0.25: 0.0~1.0 테스트 데이터셋 비율
- train_size = None: 0.0~1.0 훈련 데이터셋 비율, test_size를 작성하면 작성하지 않아도 자동 산출
- random_state = None: 정수 값, 난수 발생의 시드(seed) 값
- stratify: y의 지정한 데이터 비율을 유지(층화추출), y가 범주형일 때 사용함

A = train_test_split(X, Y, random_state=0, test_size=0.25, stratify=y)  # Scaler 미사용 시
A = train_test_split(X_scaled, Y, random_state=0, test_size=0.25, stratify=y)  # Scaler 사용 시 X_scaled 확인 필요
A = train_test_split(X_poly, Y, random_state=0, test_size=0.25)  # PolynomialFeatures 사용 시 X_scaled 확인 필요
x_train, x_test, y_train, y_test = A
[x.shape for x in [x_train, x_test, y_train, y_test]]  #데이터 분할된 비율이 제대로 됐는지 확인 목적


## 선형 모델링(분류-범주형)

In [ ]:
- max_iter = 100: Maximum number of iterations taken for the solvers to converge. Tolerance값보다 작으면 max_iter 전에 빠져나옴
- Undefitting일 경우 크게, overfitting일 경우 작게 설정
- tol = 0.0001: Tolerance for stopping criteria.
- Overfitting일 경우 크게 설정

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, tol=0.0001, random_state=None, max_iter=100)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier


## 선형 모델링(회귀-연속형)

In [ ]:
# alpha(규제 강도): Overfitting 발생 시 모델을 단순화(=규제) 시키기 위한 정도, alpha가 너무 크면 과소적합, 너무 작으면 과대적합

from sklearn.linear_model import LinearRegression  # y1 = a*x1 + b

from sklearn.linear_model import Lasso  # L1

from sklearn.linear_model import Ridge  # L2
alpha = 1 
model = Ridge(alpha=alpha).fit(x_train, y_train)

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

## 비선형 모델링

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

## 모델 평가

In [ ]:
model.score(x_train, y_train)
model.score(x_test, y_test)

## 모델 예측

In [ ]:
#실제값
y_test

#예측값
model.predict(x_test)

# 처음 5개만 Test 하고 싶을 때
y_test.iloc[:5].to_numpy() #실제값
model.predict(x_test[:5]) #에측값

# 분류 모델일 경우, 범주 별 예측 확률
model.predict_proba(x_test)  

## 성능 평가

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def print_lr_errors(model, X_test, y_test):
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)  # = mse**0.5
    r2 = r2_score(y_test, pred)
    print(f'MAE:{mae:.3f}, MSE:{mse:.3f}, RMSE:{rmse:.3f}, R2:{r2:.3f}')


print_lr_errors(model, x_train, y_train)
print_lr_errors(model, x_test, y_test)

In [ ]:
##### 범주형 - 범주형 두개의 변수는 독립일까? 관계가 있을까?
##### 카이제곱 검정
##### 귀무가설 : 두 변수의 관계는 독립이다
##### 대립가설 : 두 변수의 관계는 독립이 아니다
##### p ≤ 0.05 일때 두 변수는 독립이 아닌 관계가 있는 것
from scipy import stats

def statistical_test(mode, df, cat, target):
    uniques= df[cat].unique()    
    if mode == 't':   # 범주가 1개, 2개 일때 사용 - 범주별 평균의 차이가 유의미한가? (성별에 따른 키 평균)
        group = df.groupby(cat)[target]
        samples = [group.get_group(i) for i in uniques]
        value, p = stats.ttest_ind(*samples)        
    elif mode == 'f':   # 범주가 3개 이상일 때 사용 - 범주별 평균의 차이가 유의미한가? (혈액형에 따른 키 평균)
        group = df.groupby(cat)[target]
        samples = [group.get_group(i) for i in uniques]
        value, p = stats.f_oneway(*samples)
    elif mode == 'c':   # 두 개 범주가 독립성을 갖는지 아닌지? (가사노동의 종류와 구성원은 독립적? 연관성?)
        contingency = pd.crosstab(index=df[cat], columns=df[target])
        value, p, df, expected = stats.chi2_contingency(contingency)
        
    return value, p, 'Statistically Signifcant' if p<=0.05 else 'Statistically Insignifcant'